## Initial Setup

In [2]:
#LangChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage

#LangGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

#Pydantic and Typing
from pydantic import BaseModel
from typing import List, Dict, Any, Optional

#Utilities
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os
import uuid

In [3]:
# Load environment variables from .env file
load_dotenv()

True

## Agent Components

### LLM

In [4]:
# LLM for reasoning and tool use
llm = ChatOpenAI(
    model=os.getenv("LLM_MODEL"),
    temperature=0.1,  # Lower temperature for precision needed in diagnostics
    max_tokens=256,
    streaming=True,
    api_key=os.getenv("TFY_API_KEY"),
    base_url=os.getenv("LLM_GATEWAY_URL"),
)

### Memory

In [5]:
#Initialize State/Memory
memory = MemorySaver()

### Prompt

In [6]:
#Prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """
    You are a **Certified Sleep Therapist and ResMed Device Support Agent**. Your primary goal is to 
    help users troubleshoot their CPAP devices, check their usage compliance, and provide accurate, 
    medically compliant information.

    RULES:
    1. **Prioritize Safety:** Do not recommend any changes to therapy pressure (settings) unless explicitly 
       instructed by a doctor or in a clear troubleshooting step (e.g., pressure check).
    2. **Use Data First:** Always use your tools to check **device usage data** (leak rate, compliance) 
       before giving general advice, as the problem is often device-specific.
    3. **Be Empathetic:** Maintain a professional, reassuring, and empathetic tone.
    """
        ),
        MessagesPlaceholder(variable_name="messages", optional=True),
    ]
)

### Tools

#### Data Models

In [7]:
class DeviceMetrics(BaseModel):
    """Represents real-time metrics for a CPAP device."""
    model_name: str
    usage_hours_last_week: float
    avg_mask_leak_rate: float
    last_service_date: str

In [8]:
class DeviceData(BaseModel):
    """Simulates a user's cloud-connected device data."""
    device_metrics: List[DeviceMetrics]

    def get_all_device_models(self) -> list[str]:
        """Returns a list of all connected device model names."""
        return [m.model_name for m in self.device_metrics]

    def get_metrics_by_model(self, model_name: str) -> DeviceMetrics:
        """Returns the specific metrics for a given device model.

        Raises:
            ValueError: If the device model name is not found.
        """
        for metrics in self.device_metrics:
            if model_name.lower() == metrics.model_name.lower():
                return metrics
        
        valid_models = self.get_all_device_models()
        error_message = (
            f"Device model '{model_name}' not found. Valid models are: {', '.join(valid_models)}"
        )
        raise ValueError(error_message)

    def check_compliance(self, model_name: str) -> Dict[str, Any]:
        """Calculates compliance based on 7-day usage."""
        metrics = self.get_metrics_by_model(model_name)
        
        # Compliance requires 4 hours per night, 70% of nights (4/7 nights)
        is_compliant = metrics.usage_hours_last_week >= (4 * 7 * 0.7)
        
        return {
            "compliant": is_compliant,
            "usage": f"{metrics.usage_hours_last_week:.1f} hours last week",
            "leak_rate": f"{metrics.avg_mask_leak_rate} L/min",
            "recommendation": "High leak rate may require mask refitting." if metrics.avg_mask_leak_rate > 24 else "Usage looks stable."
        }

#### Tools

In [9]:
# Simulated live data for the user's devices
USER_DEVICES = DeviceData(
    device_metrics=[
        DeviceMetrics(model_name="AirSense 10", usage_hours_last_week=32.5, avg_mask_leak_rate=15.2, last_service_date="2025-01-15"),
        DeviceMetrics(model_name="AirMini", usage_hours_last_week=4.0, avg_mask_leak_rate=30.1, last_service_date="2024-11-01"),
    ]
)

In [10]:
@tool
def list_available_devices() -> str:
    """List all connected device model names for which data is available."""
    return USER_DEVICES.get_all_device_models()

In [11]:
@tool
def check_device_compliance(model_name: str) -> str:
    """
    Checks the user's therapy compliance metrics (usage hours and mask leak rate) 
    for a specific device model. Use the exact model name.
    """
    try:
        compliance_data = USER_DEVICES.check_compliance(model_name)
        
        response = f"Compliance Status: {'COMPLIANT' if compliance_data['compliant'] else 'NON-COMPLIANT'}."
        response += f" Usage: {compliance_data['usage']}."
        response += f" Leak Rate: {compliance_data['leak_rate']}."
        response += f" Recommendation: {compliance_data['recommendation']}"
        return response
    except ValueError as error:
        return f"{error}"

In [12]:
@tool
def find_troubleshooting_manual(device_model: str, issue_keywords: str) -> str:
    """
    Simulates searching a manual for specific issues (e.g., 'AirSense 10' and 'clicking sound').
    Returns a link or text snippet from the manual.
    """
    if "clicking" in issue_keywords.lower() and "airsense 10" in device_model.lower():
        return "Manual Page 52: Clicking sounds can be a symptom of filter blockage or water in the tubing. Please check and dry the tubing."
    
    return f"I found no specific manual page for '{issue_keywords}' on the {device_model}. Try simplifying your query."

In [13]:
tools = [list_available_devices, check_device_compliance, find_troubleshooting_manual]

## Create ReACT Agent

In [14]:
AGENT = create_react_agent(model=llm, tools=tools, state_modifier=prompt_template, checkpointer=memory)

In [15]:
def get_ai_response(events):
    for event in reversed(events):
        if event.get("messages"):
            last_message = event["messages"][-1]
            if isinstance(last_message, AIMessage) and not last_message.tool_calls:
                try:
                    content = last_message.content
                    # Simplified content handling (keeping it robust)
                    if isinstance(content, str):
                        return content
                    elif isinstance(content, list):
                        # Assuming a flat list of items for simplicity
                        return " ".join([str(item) for item in content])
                    else:
                        return str(content)
                except Exception as e:
                    print(f"Error extracting response: {e}")
                    return "An error occurred while processing the response."

    return None

In [16]:
def print_event(event):
    message = event.get("messages", [])
    if message:
        if isinstance(message, list):
            message = message[-1]
        message.pretty_print()

In [17]:
def run_agent(thread_id: str, user_input: str):
    config = {"configurable": {"thread_id": thread_id}}
    inputs = {"messages": [("user", user_input)]}

    events = []
    for event in AGENT.stream(inputs, config=config, stream_mode="values"):
        print_event(event)
        events.append(event)

    response = get_ai_response(events)
    if response is None:
        response = "An internal error has occurred."
    return {"response": response}

### Bring It Together and Test

In [18]:
thread_id = str(uuid.uuid4())

In [19]:
response = run_agent(thread_id, "I hear a clicking sound in my AirSense 10, what should I do?")

================================ Human Message =================================

I hear a clicking sound in my AirSense 10, what should I do?
================================== Ai Message ==================================
Tool Calls:
  find_troubleshooting_manual (call_xJsZ5kH9z0z2EdxSkyYWZf4O)
 Call ID: call_xJsZ5kH9z0z2EdxSkyYWZf4O
  Args:
    device_model: AirSense 10
    issue_keywords: clicking sound
================================= Tool Message =================================
Name: find_troubleshooting_manual

Manual Page 52: Clicking sounds can be a symptom of filter blockage or water in the tubing. Please check and dry the tubing.
================================== Ai Message ==================================

A clicking sound in your AirSense 10 can be caused by a few issues, such as a blocked filter or water in the tubing. Here are some steps you can take to troubleshoot the problem:

1. **Check the Filter:** Ensure that the air filter is clean and not blocked. If it's 

In [20]:
display(Markdown(response['response']))

A clicking sound in your AirSense 10 can be caused by a few issues, such as a blocked filter or water in the tubing. Here are some steps you can take to troubleshoot the problem:

1. **Check the Filter:** Ensure that the air filter is clean and not blocked. If it's dirty, replace it with a new one.

2. **Inspect the Tubing:** Look for any water in the tubing. If you find any, disconnect the tubing and allow it to dry completely before reattaching it.

3. **Reassemble:** After checking the filter and tubing, reassemble your device and see if the clicking sound persists.

If the issue continues after these checks, please let me know, and we can explore further troubleshooting options.